In [2]:
import pandas as pd 
df1=  pd.read_csv('wa_cannabis_sales - Sheet1.csv')
df2= pd.read_csv('Product Manufacturers - Retail.csv')
df3= pd.read_csv('Product Manufacturers - Medical.csv')
df4= pd.read_csv('price_per_ounce_mass_2025.csv')
df5= pd.read_csv('Colorado_Stores - Retail.csv')
df6= pd.read_csv('Colorado_Stores - Medical.csv')
df7= pd.read_csv('Colorado_Cultivations - Retail.csv')
df8= pd.read_csv('Colorado_Cultivations - Medical.csv')
df9= pd.read_csv('Colorado Marijuana_Sales_2014_To_2024_Report.xlsx - State Report.csv')
df10= pd.read_csv('ad_retail_delivery_sales_mass_2025.csv')


In [ ]:
illionis = <script type='module' src='https://public.data.illinois.gov/javascripts/api/tableau.embedding.3.latest.min.js'></script><tableau-viz id='tableau-viz' src='https://public.data.illinois.gov/t/Public/views/CannabisSalesFigures/Wholesale' width='900' height='667' toolbar='bottom' ></tableau-viz>
california = <div class='tableauPlaceholder' id='viz1747085347282' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ha&#47;HarvestReport_17064223647370&#47;HarvestReportbyCultivationType&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='HarvestReport_17064223647370&#47;HarvestReportbyCultivationType' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ha&#47;HarvestReport_17064223647370&#47;HarvestReportbyCultivationType&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1747085347282');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';} else { vizElement.style.width='100%';vizElement.style.height='2250px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
oregon = <script type='module' src='https://data.olcc.state.or.us/javascripts/api/tableau.embedding.3.latest.min.js'></script><tableau-viz id='tableau-viz' src='https://data.olcc.state.or.us/t/OLCCPublic/views/MarketDataTableau/Prices' width='800' height='850' toolbar='bottom' ></tableau-viz>

In [32]:
df1.head()

,Business Name,License Type,City,County,Sales Last Month,Period
0,Forbidden Farms,Processor,Tacoma,"Pierce county, WA","$1,420,386",Mar 2025
1,Edgemont Group,Processor,East Wenatchee,"Douglas county, WA","$1,339,568",Mar 2025
2,Lifted Cannabis,Tier None Producer & Processor,Tacoma,"Pierce county, WA","$1,197,440",Mar 2025
3,Ncmx,Tier 3 Producer & Processor,Tacoma,"Pierce county, WA","$1,060,249",Mar 2025
4,Ttl Holdings,Tier None Producer & Processor,Seattle,"King county, WA","$1,017,401",Mar 2025


In [3]:
import requests
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# ——— 1) Load & clean WA sales data ———
df = pd.read_csv('wa_cannabis_sales - Sheet1.csv')
df['Sales'] = (
    df['Sales Last Month']
      .replace(r'[\$,]', '', regex=True)
      .astype(float)
)
df['Date']    = pd.to_datetime(df['Period'], format='mixed')
df['Year']    = df['Date'].dt.year
df['Quarter'] = df['Date'].dt.quarter

df['County_clean'] = (
    df['County']
      .str.replace(r'(?i)\s*county,\s*wa$', '', regex=True)
      .str.replace(r'\s+', '', regex=True)
      .str.lower()
)

# ——— 2) Hard-coded map of WA county → FIPS ———
wa_name2fips = {
    'adams':    '53001', 'asotin':   '53003', 'benton':   '53005',
    'chelan':   '53007', 'clallam':  '53009', 'clark':    '53011',
    'columbia': '53013', 'cowlitz':  '53015', 'douglas':  '53017',
    'ferry':    '53019', 'franklin': '53021', 'garfield': '53023',
    'grant':    '53025', 'graysharbor':'53027','island':   '53029',
    'jefferson':'53031', 'king':     '53033', 'kitsap':   '53035',
    'kittitas': '53037', 'klickitat':'53039', 'lewis':    '53041',
    'lincoln':  '53043', 'mason':    '53045', 'okanogan': '53047',
    'pacific':  '53049','pendoreille':'53051','pierce':   '53053',
    'sanjuan':  '53055','skagit':   '53057','skamania': '53059',
    'snohomish':'53061','spokane':  '53063','stevens':  '53065',
    'thurston': '53067','wahkiakum':'53069','wallawalla':'53071',
    'whatcom':  '53073','whitman':  '53075','yakima':   '53077'
}
df['FIPS'] = df['County_clean'].map(wa_name2fips)
df = df[df['FIPS'].notna()].copy()

# ——— 3) Load & filter Plotly’s US-counties GeoJSON ———
us_counties = requests.get(
    "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
).json()
wa_geo = {
    "type": "FeatureCollection",
    "features": [f for f in us_counties['features'] if f['id'].startswith("53")]
}

# ——— Prepare dropdown values ———
years    = sorted(df['Year'].unique())
quarters = [1,2,3,4]

# ——— 4) Dash app layout ———
app = dash.Dash(__name__)
app.layout = html.Div(style={'padding':'1rem'}, children=[

    html.H1("WA Cannabis Sales Dashboard"),

    # Controls row
    html.Div(style={
        'display':    'flex',
        'alignItems': 'center',
        'gap':        '2rem',
        'flexWrap':   'wrap',
        'marginBottom':'1rem'
    }, children=[

        # Year dropdown
        html.Div([
            html.Label("Year"),
            dcc.Dropdown(
                id='year',
                options=[{'label': y, 'value': y} for y in years],
                value=years[-1],
                clearable=False,
                style={'width':'100px'}
            )
        ]),

        # Quarter dropdown
        html.Div([
            html.Label("Quarter"),
            dcc.Dropdown(
                id='quarter',
                options=[{'label': f"Q{q}", 'value': q} for q in quarters],
                value=quarters[0],
                clearable=False,
                style={'width':'80px'}
            )
        ]),

        # Top N slider wrapped in a Div for width
        html.Div([
            html.Label("Top N businesses"),
            html.Div(
                dcc.Slider(
                    id='top_n',
                    min=5, max=50, step=1, value=20,
                    marks={5:'5',20:'20',50:'50'},
                    tooltip={'placement':'bottom'},
                    updatemode='drag',
                    included=True
                ),
                style={'width':'300px'}   # <-- slider width
            )
        ]),

    ]),

    # Full-width map
    html.Div(
        dcc.Graph(
            id='county-map',
            style={'width':'100%','height':'600px'}  # full width
        ),
        style={'width':'100%'}
    ),

    # Two side-by-side charts
    html.Div(style={'display':'flex','gap':'1rem','marginTop':'1rem'}, children=[
        dcc.Graph(id='license-chart',  style={'flex':'1'}),
        dcc.Graph(id='business-chart', style={'flex':'1'}),
    ]),

])

# ——— 5) Callback ———
@app.callback(
    [Output('county-map','figure'),
     Output('license-chart','figure'),
     Output('business-chart','figure')],
    [Input('year','value'),
     Input('quarter','value'),
     Input('top_n','value')]
)
def update_all(year, quarter, top_n):
    dff = df[(df.Year==year)&(df.Quarter==quarter)]

    # A) Choropleth
    map_df = dff.groupby('FIPS', as_index=False)['Sales'].sum()
    fig_map = px.choropleth_mapbox(
        map_df, geojson=wa_geo,
        locations='FIPS', featureidkey='id',
        color='Sales', color_continuous_scale='Greens',
        mapbox_style='carto-positron',
        zoom=5, center={'lat':47.5,'lon':-120.7},
        opacity=0.7, labels={'Sales':'$ Sales'}
    )
    fig_map.update_layout(margin={'r':0,'t':40,'l':0,'b':0})

    # B) License Types bar
    lic = (dff.groupby('License Type', as_index=False)['Sales']
             .sum().sort_values('Sales', ascending=False))
    fig_lic = px.bar(
        lic, x='License Type', y='Sales',
        title=f"License Types — Q{quarter} {year}",
        labels={'Sales':'$ Sales'}
    )
    fig_lic.update_traces(marker_color='gold')
    fig_lic.update_layout(margin={'t':50,'b':150})
    fig_lic.update_xaxes(tickangle=-45)

    # C) Top N Businesses bar
    biz = (dff.groupby('Business Name', as_index=False)['Sales']
             .sum().sort_values('Sales', ascending=False).head(top_n))
    fig_biz = px.bar(
        biz, x='Business Name', y='Sales',
        title=f"Top {top_n} Businesses — Q{quarter} {year}",
        labels={'Sales':'$ Sales'}
    )
    fig_biz.update_traces(marker_color='saddlebrown')
    fig_biz.update_layout(margin={'t':50,'b':150})
    fig_biz.update_xaxes(tickangle=-45)

    return fig_map, fig_lic, fig_biz

if __name__ == '__main__':
    app.run(debug=True, port=8054)


In [56]:
df2.head()

,License Number,Facility Name,DBA,Facility Type,Street,City,ZIP Code,Date Updated
0,404R-00138,1-11 LLC,1:11,Retail Marijuana Products Mfg,54414 County Road F,Center,81125,2025-05-01
1,404R-00389,808 DABS LLC,NaN,Retail Marijuana Products Mfg,4550 North Jackson Street,Denver,80216,2025-05-01
2,404R-00222,81082 LLC,Colorado Cannabis Exchange II,Retail Marijuana Products Mfg,123 Pine Street,Trinidad,81082,2025-05-01
3,404R-00558,A & M LLC,Border Buds,Retail Marijuana Products Mfg,84 Goldmine Drive,Pagosa Springs,81147,2025-05-01
4,404R-00460,ACES PLACE LLC,NaN,Retail Marijuana Products Mfg,14621 County Road 98,Nunn,80648,2025-05-01


In [57]:
import pandas as pd

# load and tag
df_r = pd.read_csv("Product Manufacturers - Retail.csv")
df_r["license_type"] = "Recreational"

df_m = pd.read_csv("Product Manufacturers - Medical.csv")
df_m["license_type"] = "Medical"

# combine
df = pd.concat([df_r, df_m], ignore_index=True)


In [59]:
import pgeocode
nomi = pgeocode.Nominatim("us")
df["county_name"] = df["ZIP Code"].astype(str).str.zfill(5)\
                          .apply(lambda z: nomi.query_postal_code(z).county_name)


In [60]:
# count per county and license_type
counts = (
    df.groupby(["county_name", "license_type"])
      .size()
      .unstack(fill_value=0)
)

# pick which license_type is larger
counts["dominant"] = counts.apply(
    lambda row: "Recreational" if row["Recreational"] > row["Medical"] else "Medical",
    axis=1
)
counts = counts.reset_index()[["county_name", "dominant"]]


In [61]:
import geopandas as gpd
import requests
import io

# load GeoJSON of all U.S. counties
url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
counties = gpd.read_file(url)

# filter to Colorado (state FIPS = '08')
co_counties = counties[counties["id"].str.startswith("08")]
# the 'properties' field in this file usually holds the county name,
# but if not you'll need to bring in a separate name‐to‐FIPS table.


In [65]:
# ensure your df uses the same key: either county_name or FIPS
# if using names, you may need to standardize case/punctuation
merged = co_counties.merge(
    counts,
    left_on="NAME",   # or whatever the GeoJSON uses
    right_on="county_name",
    how="left"
)


In [2]:
# app.py
import pandas as pd
import pgeocode
import geopandas as gpd
import folium
from dash import Dash, html

# 1) Load & label your data
df_r = pd.read_csv("Product Manufacturers - Retail.csv")
df_r["license_type"] = "Recreational"

df_m = pd.read_csv("Product Manufacturers - Medical.csv")
df_m["license_type"] = "Medical"

df = pd.concat([df_r, df_m], ignore_index=True)

# 2) ZIP → county name (using pgeocode)
nomi = pgeocode.Nominatim("us")
df["ZIP Code"] = df["ZIP Code"].astype(str).str.zfill(5)
df["county_name"] = df["ZIP Code"].apply(lambda z: nomi.query_postal_code(z).county_name)

# 3) Count & pick dominant license
counts = (
    df.groupby(["county_name", "license_type"])
      .size()
      .unstack(fill_value=0)
)
counts["dominant"] = counts.apply(
    lambda r: "Recreational" if r["Recreational"] > r["Medical"] else "Medical",
    axis=1
)
counts = counts.reset_index()[["county_name", "dominant"]]

# 4) Get CO county geometries (Census shapefile or GeoJSON)
#    Here we use GeoPandas to pull down the Census shapefile for all US counties,
#    then filter to Colorado (state FIPS = '08').
#    You can replace this with a local shapefile if you prefer.
geo = gpd.read_file(
    "https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip"
)
co = geo[geo["STATEFP"] == "08"]  # Colorado

# 5) Merge counts into the GeoDataFrame
co = co.merge(counts, left_on="NAME", right_on="county_name", how="left")

# 6) Build the Folium map
m = folium.Map(location=[39.0, -105.5], zoom_start=6, tiles="cartodbpositron")
folium.GeoJson(
    co,
    style_function=lambda feat: {
        "fillColor":   "green"  if feat["properties"]["dominant"]=="Recreational"
                       else "yellow",
        "color":       "black",
        "weight":      1,
        "fillOpacity": 0.7,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NAME","dominant"],
        aliases=["County","Dominant License"]
    )
).add_to(m)

# grab the rendered HTML
map_html = m.get_root().render()

# 7) Dash app
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Colorado: Dominant Marijuana License Type by County"),
    # embed via Iframe
    html.Iframe(
        srcDoc=map_html,
        style={"width": "100%", "height": "75vh", "border": "none"}
    )
])

if __name__ == "__main__":
    app.run(debug=True,port=8055)


In [6]:
df4.head()

,CCCLastUpdated,SOLDDATE,CONSUMER_G,PATIENT_G,GRANDTOTAL_G,CONSUMER_OZ,PATIENT_OZ,GRANDTOTAL_OZ
0,07/01/2024,12/2018,14.23,10.10,13.76,403.33,286.24,390.19
1,07/01/2024,01/2019,14.54,10.21,14.16,412.12,289.36,401.48
2,07/01/2024,02/2019,14.61,10.11,14.29,414.06,286.65,405.17
3,07/01/2024,03/2019,14.55,10.42,14.23,412.40,295.48,403.39
4,07/01/2024,04/2019,14.53,10.83,13.72,411.88,307.15,388.94


In [72]:
df5.head()

,License Number,Facility Name,DBA,Facility Type,Street,City,ZIP Code,Date Updated
0,402R-00581,1-11 LLC,one:eleven,Retail Marijuana Store,17034 Highway 17,Moffat,81143,2025-05-01
1,402R-01100,1022 S ROYER LLC,Hightops,Retail Marijuana Store,1022 South Royer Street,Colorado Springs,80903,2025-05-01
2,402R-00739,10232 LLC,Provisions,Retail Marijuana Store,10232 Interstate 25,Walsenburg,81089,2025-05-01
3,402R-00587,11761013 COLORADO HOLDINGS LLC,ACA COLORADO,Retail Marijuana Store,1911 South Broadway,Denver,80210,2025-05-01
4,402R-01041,1310 FACTORY CIRCLE LLC,Nature's Herbs & Wellness Center of Fort Lupton,Retail Marijuana Store,1310 Factory Circle Unit A,Fort Lupton,80621,2025-05-01


In [8]:
df6.head()


,License Number,Facility Name,DBA,Facility Type,Street,City,ZIP Code,Date Updated
0,402-01197,1-11 LLC,one:eleven,Medical Marijuana Store,17034 Highway 17,Moffat,81143,2025-05-01
1,402-00116,1022 S ROYER LLC,Hightops,Medical Marijuana Store,1022 South Royer Street,Colorado Springs,80903,2025-05-01
2,402-00455,11761013 COLORADO HOLDINGS LLC,ACA COLORADO,Medical Marijuana Store,1911 South Broadway,Denver,80210,2025-05-01
3,402-00248,2306 N POWERS LLC,Hightops,Medical Marijuana Store,"2306 North Powers Frontage Road, Suite 100",Colorado Springs,80915,2025-05-01
4,402-00618,4 ROSES LLC,Bleezies,Medical Marijuana Store,3234 North Nevada Avenue,Colorado Springs,80907,2025-05-01


In [9]:
df7.head()

,License Number,Facility Name,DBA,Facility Type,Street,City,ZIP Code,Date Updated
0,403R-00537,1-11 LLC,1:11,Retail Marijuana Cultivation Facility,54414 County Road F,Center,81125,2025-05-01
1,403R-00974,132 ENTERPRISES LLC,KLONE; RadCanna,Retail Marijuana Cultivation Facility,"322 North Norfolk Street, Unit I",Aurora,80011,2025-05-01
2,403R-01454,1395 SOUTH ACOMA LLC,Stellar Decon; Stellar Cannabis,Retail Marijuana Cultivation Facility,1395 South Acoma Street,Denver,80223,2025-05-01
3,403R-00885,1790 POPE VALLEY LLC,Canna Tek,Retail Marijuana Cultivation Facility,1790 Pope Valley Ranch Road,Pueblo,81005,2025-05-01
4,403R-01401,1CURE LLC,NaN,Retail Marijuana Cultivation Facility,7835 Maverick Lane,Ordway,81063,2025-05-01


In [10]:
df8.head()

,License Number,Facility Name,DBA,Facility Type,Street,City,ZIP Code,Date Updated
0,403-01840,1-11 LLC,one:eleven,Medical Marijuana Cultivation,54414 County Road F,Center,81125,2025-05-01
1,403-00164,1022 S ROYER LLC,Hightops,Medical Marijuana Cultivation,1022 South Royer Street,Colorado Springs,80903,2025-05-01
2,403-00687,11761013 COLORADO HOLDINGS LLC,ACA COLORADO,Medical Marijuana Cultivation,3410 Astrozon Court,Colorado Springs,80910,2025-05-01
3,403-01932,132 ENTERPRISES LLC,KLONE; RadCanna,Medical Marijuana Cultivation,322 North Norfolk Street Unit I,Aurora,80011,2025-05-01
4,403-01080,1395 SOUTH ACOMA LLC,Stellar Decon; Stellar Cannabis,Medical Marijuana Cultivation,1395 South Acoma Street,Denver,80223,2025-05-01


In [11]:
df9.head()

,Marijuana Sales Report\nJanuary 2014 to December 2024,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Intentionally left blank,NaN,NaN,NaN,NaN
1,Month,Year,Total Medical \nMarijuana Sales ¹,Total Retail \nMarijuana Sales ²,Total \nMarijuana Sales
2,12,2024,"$12,369,108","$101,207,275","$113,576,383"
3,11,2024,"$11,878,351","$97,379,486","$109,257,837"
4,10,2024,"$12,416,284","$100,887,426","$113,303,710"


In [12]:
df10.head()

,SaleDate,UnitOfMeasureName,CountBasedTotal,DollarCountBasedTotal,WeightBasedTotal,DollarWeightBasedTotal,Quantity,Units,TOTAL_$,ProductCategoryName,ProductCategoryTypeName
0,4/21/2025,mg,0,0.00,10800.00,66.16,10800.00,6,66.16,Infused Pre-Rolls,Concentrate
1,4/21/2025,ea,3193,18949.75,0.00,0.00,3193.00,1723,18949.75,Infused Beverage,InfusedEdible
2,4/21/2025,lb,0,0.00,0.36,250.00,0.36,7,250.00,Shake/Trim,ShakeTrim
3,4/21/2025,ea,6,157.50,0.00,0.00,6.00,6,157.50,Suppository,InfusedNonEdible
4,4/21/2025,g,0,0.00,1727.11,28356.07,1727.11,864,28356.07,Concentrate,Concentrate
